In [1]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.parent.resolve()))
dllpath = Path("C:") /"Program Files" / "NVIDIA GPU Computing Toolkit" / "CUDA" / "v11.2" / "bin" 
dllstring = str(dllpath.resolve())
os.add_dll_directory(dllstring)

<AddedDllDirectory('C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.2\\bin')>

In [2]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [3]:
from sources.dataset_utils.get_iid_dataset_utils import get_full_iid_dataset, \
    get_default_iid_dataset
from sources.datasets.femnist.femnist_client_dataset import FemnistClientDataset
from sources.datasets.shakespeare.shakespeare_client_dataset import ShakespeareClientDataset
from sources.flwr_parameters.set_random_seeds import DEFAULT_SEED
from sources.metrics.default_metrics import DEFAULT_METRICS
from sources.metrics.sparse_metric_decorator import SparseMetricDecorator
from sources.models.femnist.femnist_model_template import FemnistModelTemplate
import tensorflow_addons as tfa
from sources.models.shakespeare.shakespeare_model_template import ShakespeareModelTemplate

In [4]:
data_dir = Path(os.getcwd()).parent.parent / "data"

In [5]:
model_template = FemnistModelTemplate(DEFAULT_SEED)
model = model_template.get_model()
dataset_v = FemnistClientDataset(str(data_dir), "1")

In [6]:
dataset = get_full_iid_dataset("femnist")

In [7]:
model = model_template.get_model()
optimizer = model_template.get_optimizer(0.001)
loss = model_template.get_loss()

In [8]:
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy", 
                       SparseMetricDecorator(tfa.metrics.MatthewsCorrelationCoefficient(model_template.num_classes)), 
                       SparseMetricDecorator(tfa.metrics.F1Score(model_template.num_classes, name="macro_f1_score", average='macro')),
                       SparseMetricDecorator( tfa.metrics.F1Score(model_template.num_classes, name="micro_f1_score", average='micro'))
                       ])

In [19]:
history = model.fit(
        epochs=100,
        x=dataset_v.process_x(dataset.train["x"]),
        y=dataset_v.process_y(dataset.train["y"]),
        steps_per_epoch=100,
        batch_size=32,
        shuffle=True,
        validation_steps=30,
        validation_data = (dataset_v.process_x(dataset.test["x"]),
                           dataset_v.process_y(dataset.test["y"]))
)

Epoch 1/100
100/100 [==============================] - 3s 18ms/step - loss: 5.1414 - accuracy: 0.1391 - MatthewsCorrelationCoefficient: 0.0888 - macro_f1_score: 0.0270 - micro_f1_score: 0.1391 - val_loss: 3.5974 - val_accuracy: 0.2302 - val_MatthewsCorrelationCoefficient: 0.1922 - val_macro_f1_score: 0.0442 - val_micro_f1_score: 0.2302
Epoch 2/100
100/100 [==============================] - 1s 12ms/step - loss: 2.9301 - accuracy: 0.3094 - MatthewsCorrelationCoefficient: 0.2741 - macro_f1_score: 0.0707 - micro_f1_score: 0.3094 - val_loss: 3.2611 - val_accuracy: 0.3938 - val_MatthewsCorrelationCoefficient: 0.3652 - val_macro_f1_score: 0.0996 - val_micro_f1_score: 0.3938
Epoch 3/100
100/100 [==============================] - 1s 12ms/step - loss: 2.4779 - accuracy: 0.4353 - MatthewsCorrelationCoefficient: 0.4067 - macro_f1_score: 0.1242 - micro_f1_score: 0.4353 - val_loss: 2.8870 - val_accuracy: 0.4719 - val_MatthewsCorrelationCoefficient: 0.4475 - val_macro_f1_score: 0.1498 - val_micro_f1_

100/100 [==============================] - 1s 13ms/step - loss: 0.8109 - accuracy: 0.7769 - MatthewsCorrelationCoefficient: 0.7670 - macro_f1_score: 0.4327 - micro_f1_score: 0.7769 - val_loss: 1.6469 - val_accuracy: 0.6917 - val_MatthewsCorrelationCoefficient: 0.6793 - val_macro_f1_score: 0.3396 - val_micro_f1_score: 0.6917
Epoch 26/100
100/100 [==============================] - 1s 13ms/step - loss: 0.7653 - accuracy: 0.7859 - MatthewsCorrelationCoefficient: 0.7758 - macro_f1_score: 0.4194 - micro_f1_score: 0.7859 - val_loss: 1.6655 - val_accuracy: 0.6938 - val_MatthewsCorrelationCoefficient: 0.6817 - val_macro_f1_score: 0.3742 - val_micro_f1_score: 0.6938
Epoch 27/100
100/100 [==============================] - 1s 13ms/step - loss: 0.7577 - accuracy: 0.7909 - MatthewsCorrelationCoefficient: 0.7816 - macro_f1_score: 0.4492 - micro_f1_score: 0.7909 - val_loss: 1.4969 - val_accuracy: 0.7208 - val_MatthewsCorrelationCoefficient: 0.7093 - val_macro_f1_score: 0.3947 - val_micro_f1_score: 0.7

100/100 [==============================] - 1s 13ms/step - loss: 0.7993 - accuracy: 0.7709 - MatthewsCorrelationCoefficient: 0.7605 - macro_f1_score: 0.4206 - micro_f1_score: 0.7709 - val_loss: 1.5199 - val_accuracy: 0.6948 - val_MatthewsCorrelationCoefficient: 0.6820 - val_macro_f1_score: 0.3781 - val_micro_f1_score: 0.6948
Epoch 50/100
100/100 [==============================] - 1s 13ms/step - loss: 0.7298 - accuracy: 0.7909 - MatthewsCorrelationCoefficient: 0.7814 - macro_f1_score: 0.4298 - micro_f1_score: 0.7909 - val_loss: 1.5101 - val_accuracy: 0.7063 - val_MatthewsCorrelationCoefficient: 0.6941 - val_macro_f1_score: 0.3910 - val_micro_f1_score: 0.7063
Epoch 51/100
100/100 [==============================] - 1s 13ms/step - loss: 0.6619 - accuracy: 0.8103 - MatthewsCorrelationCoefficient: 0.8015 - macro_f1_score: 0.4601 - micro_f1_score: 0.8103 - val_loss: 1.5182 - val_accuracy: 0.7010 - val_MatthewsCorrelationCoefficient: 0.6888 - val_macro_f1_score: 0.3747 - val_micro_f1_score: 0.7

100/100 [==============================] - 1s 13ms/step - loss: 0.4983 - accuracy: 0.8481 - MatthewsCorrelationCoefficient: 0.8413 - macro_f1_score: 0.5194 - micro_f1_score: 0.8481 - val_loss: 1.3774 - val_accuracy: 0.7469 - val_MatthewsCorrelationCoefficient: 0.7369 - val_macro_f1_score: 0.4394 - val_micro_f1_score: 0.7469
Epoch 74/100
100/100 [==============================] - 1s 13ms/step - loss: 0.5016 - accuracy: 0.8466 - MatthewsCorrelationCoefficient: 0.8396 - macro_f1_score: 0.5081 - micro_f1_score: 0.8466 - val_loss: 1.3533 - val_accuracy: 0.7427 - val_MatthewsCorrelationCoefficient: 0.7324 - val_macro_f1_score: 0.4398 - val_micro_f1_score: 0.7427
Epoch 75/100
100/100 [==============================] - 1s 13ms/step - loss: 0.5076 - accuracy: 0.8600 - MatthewsCorrelationCoefficient: 0.8538 - macro_f1_score: 0.5248 - micro_f1_score: 0.8600 - val_loss: 1.3329 - val_accuracy: 0.7531 - val_MatthewsCorrelationCoefficient: 0.7432 - val_macro_f1_score: 0.4527 - val_micro_f1_score: 0.7

100/100 [==============================] - 1s 13ms/step - loss: 0.4610 - accuracy: 0.8622 - MatthewsCorrelationCoefficient: 0.8558 - macro_f1_score: 0.5236 - micro_f1_score: 0.8622 - val_loss: 1.3299 - val_accuracy: 0.7542 - val_MatthewsCorrelationCoefficient: 0.7440 - val_macro_f1_score: 0.4670 - val_micro_f1_score: 0.7542
Epoch 98/100
100/100 [==============================] - 1s 13ms/step - loss: 0.4531 - accuracy: 0.8616 - MatthewsCorrelationCoefficient: 0.8552 - macro_f1_score: 0.5359 - micro_f1_score: 0.8616 - val_loss: 1.3094 - val_accuracy: 0.7635 - val_MatthewsCorrelationCoefficient: 0.7541 - val_macro_f1_score: 0.4783 - val_micro_f1_score: 0.7635
Epoch 99/100
100/100 [==============================] - 1s 13ms/step - loss: 0.4386 - accuracy: 0.8625 - MatthewsCorrelationCoefficient: 0.8564 - macro_f1_score: 0.5689 - micro_f1_score: 0.8625 - val_loss: 1.3218 - val_accuracy: 0.7563 - val_MatthewsCorrelationCoefficient: 0.7466 - val_macro_f1_score: 0.4802 - val_micro_f1_score: 0.7

In [11]:
model.evaluate(
    x=dataset_v.process_x(dataset.validation["x"][0:5]),
    y=dataset_v.process_y(dataset.validation["y"][0:5]),
    return_dict=True
)

1/1 [==============================] - 0s 54ms/step - loss: 65.1980 - accuracy: 0.0000e+00 - MatthewsCorrelationCoefficient: 0.0000e+00 - macro_f1_score: 0.0000e+00 - micro_f1_score: 0.0000e+00


{'loss': 65.19803619384766,
 'accuracy': 0.0,
 'MatthewsCorrelationCoefficient': 0.0,
 'macro_f1_score': 0.0,
 'micro_f1_score': 0.0}

In [21]:
model_template = ShakespeareModelTemplate(DEFAULT_SEED)
model = model_template.get_model()
dataset_v = ShakespeareClientDataset(str(data_dir), "1")

In [22]:
dataset = get_full_iid_dataset("shakespeare")

In [23]:
model = model_template.get_model()
optimizer = model_template.get_optimizer(0.01)
loss = model_template.get_loss()

In [24]:
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy", 
                       tfa.metrics.MatthewsCorrelationCoefficient(model_template.num_classes), 
                       tfa.metrics.F1Score(model_template.num_classes, 
                                           name="macro_f1_score", average='macro'),
                        tfa.metrics.F1Score(model_template.num_classes, 
                                            name="micro_f1_score", average='micro')])

In [ ]:
history = model.fit(
    epochs=100,
        x=dataset_v.process_x(dataset.train["x"]),
          y=dataset_v.process_y(dataset.train["y"]),
    steps_per_epoch=100,
        batch_size=32,
        shuffle=True,
        validation_steps=15,
)

In [14]:
import pickle as pkl

path = Path("I:\Projects\impact_of_non_iid_data_in_federated_learning\checkpoints"
            "\CentralisedEvaluationFemnist1\FedAvg_SGD_lr0.01_nr5_nc600_le1_i0\FedAvg_SGD_lr0.01_nr5_nc600_le1_i0_run_0\metrics\centralised_evaluation_metrics_CentralisedEvaluationFemnist1_5.pkl")
with path.open("rb") as f:
    data=pkl.load(f)
    

In [15]:
data



{'loss': 68.79035949707031,
 'accuracy': 0.01862373761832714,
 'MatthewsCorrelationCoefficient': 0.004405854735523462,
 'macro_f1_score': 0.001531881163828075,
 'micro_f1_score': 0.01862373761832714}